# ETL Pipeline

In [74]:
# Importing libraries
import pandas as pd
import numpy as np
import sqlite3

## Assess

### Messages Data

In [75]:
# Loading data
messages = pd.read_csv('data/disaster_messages.csv')
messages.sample(10)

,id,message,original,genre
23710,27408,Lower cotton production will also impact on th...,NaN,news
10628,12191,"I have children 's clothing , adult clothing ,...",NaN,direct
15959,18787,"The local office of the Meteorology, Climatolo...",NaN,news
19683,22921,"Eveline Herfkens, Minister for Development Coo...",NaN,news
13910,16537,"Contrary to the arrival of a warmer season, th...",NaN,news
26011,29999,A Child Friendly Environment Drive is now unde...,NaN,news
9193,10323,"if they have cyclone,what should do?",Si cyclone nan ta ki sa pou fe,direct
3466,3984,There are some people that I know in central P...,Gen kek moun mwen konnen ki sit potoprens blik...,direct
15165,17916,Officials in the Disaster Management Ministry ...,NaN,news
10837,12436,sooo I 'm sitting in my living room .. doing n...,NaN,social


In [76]:
# Unique values
messages.genre.unique()

array(['direct', 'social', 'news'], dtype=object)

In [77]:
#Shape of the data
messages.shape

(26248, 4)

In [78]:
# Checking null values
messages.isna().sum()

id              0
message         0
original    16064
genre           0
dtype: int64

**Oberservations:** 

- Original Column can be droped

- If the `genre` column is to be used, it will need to be converted into dummy variables.

### Categories Data

In [79]:
categories = pd.read_csv('data/disaster_categories.csv')
categories.sample(10)

,id,categories
23527,27197,related-1;request-0;offer-0;aid_related-1;medi...
25782,29733,related-1;request-0;offer-0;aid_related-1;medi...
25939,29911,related-1;request-0;offer-0;aid_related-1;medi...
128,159,related-1;request-0;offer-0;aid_related-1;medi...
5103,5809,related-1;request-0;offer-0;aid_related-0;medi...
17339,20328,related-1;request-0;offer-0;aid_related-0;medi...
9356,10505,related-1;request-0;offer-0;aid_related-0;medi...
1134,1358,related-1;request-1;offer-0;aid_related-1;medi...
13149,15690,related-1;request-0;offer-0;aid_related-0;medi...
7673,8624,related-1;request-0;offer-0;aid_related-0;medi...


In [80]:
categories.shape

(26248, 2)

In [81]:
# Reviewing contents of categories column
print(categories.categories.iloc[0])

related-1;request-0;offer-0;aid_related-0;medical_help-0;medical_products-0;search_and_rescue-0;security-0;military-0;child_alone-0;water-0;food-0;shelter-0;clothing-0;money-0;missing_people-0;refugees-0;death-0;other_aid-0;infrastructure_related-0;transport-0;buildings-0;electricity-0;tools-0;hospitals-0;shops-0;aid_centers-0;other_infrastructure-0;weather_related-0;floods-0;storm-0;fire-0;earthquake-0;cold-0;other_weather-0;direct_report-0


**Total Categories**

In [82]:
len(categories.categories.iloc[0].split(';'))

36

**Observations:**

- To prepare for machine learning  process the categories column needs to be split into 36 columns

- Once the categories have been joined to the messages, it would be possible to drop the id column

## Cleaning Data

In [83]:
# Cleaning data
messages_clean = messages.copy()
categories_clean = categories.copy()

### Categories

**Creating new dataframe of categories with category name for heading and 1s and 0s for values**

- Use `.str.split(';', expand = True)` to split data
- Use `.str[:-2]` to get headings
- Use `.str[-1]` and `.astype(int)` to convert data to 1s and 0s

In [84]:
categories_clean.categories.str.split(';', expand = True)

,0,1,2,3,4,5,6,7,8,9,...,26,27,28,29,30,31,32,33,34,35
0,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
1,related-1,request-0,offer-0,aid_related-1,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-1,floods-0,storm-1,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
2,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
3,related-1,request-1,offer-0,aid_related-1,medical_help-0,medical_products-1,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
4,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26243,related-0,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
26244,related-0,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
26245,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
26246,related-1,request-0,offer-0,aid_related-1,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-1,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0


In [85]:
# Creating dataframe
data = categories_clean.categories.str.split(';', expand = True)

# Confirm 36 columns created
assert data.shape[1] == 36, "data columns: {}".format(data.shape[1])

In [86]:
data.iloc[0].str[:-2]

0                    related
1                    request
2                      offer
3                aid_related
4               medical_help
5           medical_products
6          search_and_rescue
7                   security
8                   military
9                child_alone
10                     water
11                      food
12                   shelter
13                  clothing
14                     money
15            missing_people
16                  refugees
17                     death
18                 other_aid
19    infrastructure_related
20                 transport
21                 buildings
22               electricity
23                     tools
24                 hospitals
25                     shops
26               aid_centers
27      other_infrastructure
28           weather_related
29                    floods
30                     storm
31                      fire
32                earthquake
33                      cold
34            

In [87]:
# Get headings
col_names = data.iloc[0].str[:-2]
data.columns = col_names
data.columns

Index(['related', 'request', 'offer', 'aid_related', 'medical_help',
       'medical_products', 'search_and_rescue', 'security', 'military',
       'child_alone', 'water', 'food', 'shelter', 'clothing', 'money',
       'missing_people', 'refugees', 'death', 'other_aid',
       'infrastructure_related', 'transport', 'buildings', 'electricity',
       'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure',
       'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold',
       'other_weather', 'direct_report'],
      dtype='object', name=0)

In [88]:
# Convert columns to int
for col in col_names:
    data[col] = data[col].str[-1].astype(int)

# Confirm 36 int columns
assert (data.dtypes == int).sum() == 36, df.dtypes

In [89]:
data

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26243,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26244,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26245,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26246,1,0,0,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [90]:
data.sum(axis = 1).sum(), data.shape[0]

(83434, 26248)

In [91]:
data.sum(axis = 1).sum()

83434

In [92]:
data.columns

Index(['related', 'request', 'offer', 'aid_related', 'medical_help',
       'medical_products', 'search_and_rescue', 'security', 'military',
       'child_alone', 'water', 'food', 'shelter', 'clothing', 'money',
       'missing_people', 'refugees', 'death', 'other_aid',
       'infrastructure_related', 'transport', 'buildings', 'electricity',
       'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure',
       'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold',
       'other_weather', 'direct_report'],
      dtype='object', name=0)

In [93]:
data[data.sum(axis = 1) != 1]

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,1,0,0,0,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
7,1,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26239,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
26241,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26243,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26244,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


**Replace `categories` column with data**
- Use `pd.concat([categories_clean, df], axis=1)` & `.drop('categories', axis=1)`

In [97]:
categories_clean.shape

(26248, 2)

In [98]:
data.shape

(26248, 36)

In [99]:
pd.concat([categories_clean.drop('categories', axis = 1), data], axis = 1)

,id,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,1,0,0,1,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,1,1,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,12,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26243,30261,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26244,30262,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26245,30263,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26246,30264,1,0,0,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [100]:
categories_clean = pd.concat([categories_clean.drop('categories', axis = 1), data], axis = 1)

# Confirm total of 37 columns
assert categories_clean.shape[1] == 37, "Categories shape: {}".format(categories_clean.shape[1])

**Create merged dataset**
- Use `.merge(on='id')`

The `original` column and `id` column were retained for storage in the database even if these will not be used in the machine learning process.

In [104]:
data_merged = messages_clean.merge(categories_clean, on = 'id')

# Confirm correct new shape
assert data_merged.shape[1] == messages_clean.shape[1] + categories_clean.shape[1] - 1, \
    "Merged df columns: {}, Combined columns: {}".format(data_merged.shape[1], 
                                                         messages_clean.shape[1] + categories_clean.shape[1] - 1)
assert data_merged.shape[0] == messages_clean.shape[0], "Merged df rows: {}, Messages df rows: {}"\
    .format(data_merged.shape[0], messages_clean.shape[0])

AssertionError: Merged df rows: 26386, Messages df rows: 26248

**This might have created duplicates by joining them by id**

In [106]:
data_merged[data_merged.duplicated()].shape

(170, 40)

There are 170 duplicated rows that need to be dropped.

Some values are not even 0 or 1.

**Drop duplicated rows.**
- Use `.drop_duplicates(inplace=True)`

In [108]:
data_merged.drop_duplicates(inplace = True)

# Confirm no duplicates
assert data_merged.duplicated().sum() == 0

**Explore categories values**

In [109]:
for col in data_merged.columns[4:]:
    if (data_merged[col] > 1).sum() > 0:
        print(col)
        print(data_merged[col].unique())
        print('\n')

related
[1 0 2]




It's just the `related` column that has these three different values. For the purposes of simply predicting related or not it is preferrable to only have "on/off" conditions. As a result, any value of 2 should be converted to 1.

In [110]:
data_merged.loc[(data_merged.related == 2), 'related'] = 1
# Confirm only two values
assert len(data_merged.related.unique()) == 2, data.related.unique()

### Loading

In [112]:
conn = sqlite3.connect('disaster_messages.db')
data_merged.to_sql('messages', con = conn, if_exists = 'replace', index = False)
conn.commit()
conn.close()

### Shape of Database

In [113]:
conn = sqlite3.connect('data/disaster_messages.db')

In [114]:
rows = pd.read_sql('SELECT COUNT(*) FROM messages', conn)

In [115]:
rows.iloc[0][0]

26216

In [116]:
messages.drop_duplicates().shape

(26180, 4)

In [117]:
categories.drop_duplicates().shape

(26216, 2)

In [118]:
pd.read_sql('SELECT message FROM messages LIMIT 5;', conn)

,message
0,Weather update - a cold front from Cuba that c...
1,Is the Hurricane over or is it not over
2,Looking for someone but no name
3,UN reports Leogane 80-90 destroyed. Only Hospi...
4,"says: west side of Haiti, rest of the country ..."


In [119]:
conn.close()